## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file.
file_path = os.path.join("..", "Weather_Database","WeatherPy_Database.csv")
city_data_df = pd.read_csv(file_path)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kirkuk,IQ,35.4681,44.3922,93.92,16,0,7.20,clear sky
1,1,Rikitea,PF,-23.1203,-134.9692,72.21,65,25,21.72,scattered clouds
2,2,Xuddur,SO,4.1213,43.8894,85.62,35,99,14.50,overcast clouds
3,3,Codrington,AU,-38.2667,141.9667,51.21,84,96,20.45,overcast clouds
4,4,Hermanus,ZA,-34.4187,19.2345,59.81,78,0,3.60,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Xuddur,SO,4.1213,43.8894,85.62,35,99,14.50,overcast clouds
5,5,Shingu,JP,33.7333,135.9833,76.01,91,77,4.45,broken clouds
6,6,Pachino,IT,36.7186,15.0907,81.25,79,25,14.16,scattered clouds
21,21,Hithadhoo,MV,-0.6000,73.0833,81.86,71,20,10.11,few clouds
31,31,Djambala,CG,-2.5447,14.7533,78.24,61,99,4.43,overcast clouds
34,34,Namibe,AO,-15.1961,12.1522,75.83,73,0,16.11,clear sky
35,35,Ribeira Grande,PT,38.5167,-28.7000,79.20,69,40,24.16,scattered clouds
37,37,Morant Bay,JM,17.8815,-76.4093,89.71,64,3,8.99,clear sky
38,38,Rosh Pinna,IL,32.9689,35.5426,79.74,45,0,7.36,clear sky
41,41,Hilo,US,19.7297,-155.0900,75.27,90,0,4.00,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                261
City                   261
Country                256
Lat                    261
Lng                    261
Max Temp               261
Humidity               261
Cloudiness             261
Wind Speed             261
Current Description    261
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [29]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = None
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Xuddur,SO,85.62,overcast clouds,4.1213,43.8894,None
5,Shingu,JP,76.01,broken clouds,33.7333,135.9833,None
6,Pachino,IT,81.25,scattered clouds,36.7186,15.0907,None
21,Hithadhoo,MV,81.86,few clouds,-0.6000,73.0833,None
31,Djambala,CG,78.24,overcast clouds,-2.5447,14.7533,None
34,Namibe,AO,75.83,clear sky,-15.1961,12.1522,None
35,Ribeira Grande,PT,79.20,scattered clouds,38.5167,-28.7000,None
37,Morant Bay,JM,89.71,clear sky,17.8815,-76.4093,None
38,Rosh Pinna,IL,79.74,clear sky,32.9689,35.5426,None
41,Hilo,US,75.27,clear sky,19.7297,-155.0900,None


In [30]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... Skipping.")     

Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.


In [33]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [36]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [39]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}, {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [40]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))